# Julia 深度學習：類神經網路模型簡介

## 作業 032：訓練 MLP 學習門牌號碼資料集

訓練一個 MLP 模型來學習門牌號碼資料集。

注意：MLP 模型的能力有限，可能會導致訓練起來效果不佳。

注意：近期 Flux 正在持續更新，請確保您的 Julia 在 v1.3 版以上，以及 Flux 在 v0.10.4 以上或是最新版。

In [1]:
using Flux
using Flux.Data: DataLoader
using Flux: @epochs, onecold, onehotbatch, throttle, logitcrossentropy
using MLDatasets
using Statistics

## 讀取資料

呼叫 SVHN2 資料集的過程中，會先去檢查以前是否有下載過，如果是第一次下載，過程中會詢問是否下載資料集，請回答 `y`。整個資料集的大小約為 1.6 GB，下載時間可能會稍久，請耐心等候。

In [21]:
train_X, train_y = SVHN2.traindata(Float32)
test_X,  test_y  = SVHN2.testdata(Float32)

(Float32[0.14901961 0.15294118 … 0.19607843 0.1882353; 0.15294118 0.15294118 … 0.2 0.1882353; … ; 0.16470589 0.16862746 … 0.1764706 0.17254902; 0.15294118 0.15294118 … 0.16470589 0.16470589]

Float32[0.40392157 0.40784314 … 0.45882353 0.4509804; 0.40784314 0.40784314 … 0.4627451 0.4509804; … ; 0.40392157 0.39607844 … 0.45490196 0.4509804; 0.38039216 0.38039216 … 0.44313726 0.44313726]

Float32[0.23529412 0.23921569 … 0.29803923 0.2901961; 0.23921569 0.23921569 … 0.3019608 0.2901961; … ; 0.24313726 0.24705882 … 0.28235295 0.2784314; 0.22352941 0.22352941 … 0.27058825 0.2784314]

Float32[0.5058824 0.5254902 … 0.5411765 0.5137255; 0.49803922 0.52156866 … 0.50980395 0.47843137; … ; 0.48235294 0.49411765 … 0.39607844 0.43529412; 0.48235294 0.49019608 … 0.4392157 0.48235294]

Float32[0.5568628 0.5882353 … 0.59607846 0.5686275; 0.56078434 0.58431375 … 0.5647059 0.53333336; … ; 0.5254902 0.5372549 … 0.41960785 0.4627451; 0.5294118 0.5372549 … 0.4627451 0.50980395]

Float32[0.6 0.627451 … 0.647

In [22]:
train_X

32×32×3×73257 Array{Float32,4}:
[:, :, 1, 1] =
 0.129412   0.109804  0.156863   0.188235   …  0.337255   0.329412   0.333333
 0.0588235  0.054902  0.0705882  0.0745098     0.341176   0.337255   0.34902 
 0.0588235  0.054902  0.0627451  0.0901961     0.321569   0.321569   0.333333
 0.164706   0.168627  0.117647   0.0901961     0.313726   0.305882   0.333333
 0.262745   0.270588  0.25098    0.231373      0.243137   0.317647   0.317647
 0.266667   0.231373  0.227451   0.262745   …  0.0823529  0.180392   0.282353
 0.266667   0.231373  0.243137   0.239216      0.0784314  0.0901961  0.121569
 0.290196   0.298039  0.301961   0.305882      0.341176   0.352941   0.352941
 0.305882   0.32549   0.329412   0.34902       0.419608   0.427451   0.439216
 0.345098   0.345098  0.34902    0.352941      0.411765   0.403922   0.415686
 0.329412   0.321569  0.32549    0.329412   …  0.415686   0.4        0.396078
 0.309804   0.329412  0.32549    0.333333      0.411765   0.407843   0.423529
 0.309804   0.329

In [23]:
train_X = Flux.flatten(train_X)
test_X = Flux.flatten(test_X)
train_y = onehotbatch(train_y, 1:10)
test_y = onehotbatch(test_y, 1:10)

10×26032 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  0  1  0  0  1  0  1  1  0  0  0  0  …  1  0  1  0  1  0  0  0  0  0  0  0
 0  1  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  1  1  0  0  0
 0  0  0  0  0  0  0  0  0  0  1  0  0     0  0  0  0  0  1  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  1     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  1  0  0  0  0  0  0  1  0  …  0  0  0  0  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  1  0  1
 0  0  0  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  1  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  0  0  0  0  0

In [24]:
batchsize = 1024
train = DataLoader(train_X, train_y, batchsize=batchsize, shuffle=true)
test = DataLoader(test_X, test_y, batchsize=batchsize)

DataLoader((Float32[0.14901961 0.5058824 … 0.3764706 0.39607844; 0.15294118 0.49803922 … 0.38039216 0.39215687; … ; 0.2784314 0.5647059 … 0.2901961 0.19607843; 0.2784314 0.6117647 … 0.26666668 0.19607843], Bool[0 0 … 0 0; 0 1 … 0 0; … ; 0 0 … 0 0; 0 0 … 0 0]), 1024, 26032, true, 26032, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  26023, 26024, 26025, 26026, 26027, 26028, 26029, 26030, 26031, 26032], false)

## model


In [30]:
model = Chain(
    Dense(1024*3, 1024, relu),
    Dense(1024, 512, relu),
    Dense(512, 10),
    softmax
)

Chain(Dense(3072, 1024, relu), Dense(1024, 512, relu), Dense(512, 10), softmax)

# loss function

In [31]:
loss(x, y) = logitcrossentropy(model(x), y)

loss (generic function with 1 method)

# callback function

In [32]:
function test_loss()
    l = 0f0
    for (x, y) in test
        l += loss(x, y)
    end
    l / length(test)  # return
end

test_loss (generic function with 1 method)

In [33]:
evalcb() = @show(test_loss())

evalcb (generic function with 1 method)

# train model

In [34]:
epochs = 20
@epochs epochs Flux.train!(loss, params(model), train, ADAM(0.005), cb=throttle(evalcb, 10))

┌ Info: Epoch 1
└ @ Main C:\Users\aband\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.3022776f0
test_loss() = 2.2648966f0


┌ Info: Epoch 2
└ @ Main C:\Users\aband\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


┌ Info: Epoch 3
└ @ Main C:\Users\aband\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


...

┌ Info: Epoch 15
└ @ Main C:\Users\aband\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2648966f0
test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


┌ Info: Epoch 16
└ @ Main C:\Users\aband\.julia\packages\Flux\Fj3bt\src\optimise\train.jl:121


test_loss() = 2.2648966f0
test_loss() = 2.2648966f0


InterruptException: InterruptException:

# SVM try

In [35]:
using LIBSVM

In [36]:
# 一般機器學習只能吃序列型態

train_X = Matrix(reshape(train_X, 32*32, :)')
test_X = Matrix(reshape(test_X, 32*32, :)')

78096×1024 Array{Float32,2}:
 0.14902   0.152941  0.152941  0.160784  …  0.168627  0.172549  0.164706
 0.403922  0.407843  0.407843  0.407843     0.454902  0.45098   0.443137
 0.235294  0.239216  0.243137  0.247059     0.278431  0.278431  0.278431
 0.505882  0.498039  0.490196  0.490196     0.396078  0.435294  0.482353
 0.556863  0.560784  0.560784  0.556863     0.423529  0.462745  0.509804
 0.6       0.596078  0.592157  0.588235  …  0.529412  0.564706  0.611765
 0.588235  0.588235  0.596078  0.603922     0.596078  0.596078  0.596078
 0.627451  0.639216  0.658824  0.682353     0.658824  0.658824  0.658824
 0.662745  0.666667  0.67451   0.686275     0.709804  0.713726  0.717647
 0.509804  0.505882  0.498039  0.486275     0.462745  0.501961  0.537255
 0.552941  0.54902   0.541176  0.533333  …  0.52549   0.564706  0.6     
 0.627451  0.623529  0.623529  0.619608     0.588235  0.623529  0.658824
 0.694118  0.694118  0.690196  0.690196     0.698039  0.686275  0.682353
 ⋮                    

In [38]:
train_y = ["$x" for x in train_y]
test_y = ["$x" for x in test_y];

In [39]:
# train

model_svm = LIBSVM.fit!(SVC(), train_X, train_y)

MethodError: MethodError: no method matching fit!(::SVC, ::Array{Float32,2}, ::Array{String,2})
Closest candidates are:
  fit!(::Union{LIBSVM.AbstractSVC, LIBSVM.AbstractSVR}, ::AbstractArray{T,2} where T) at C:\Users\aband\.julia\packages\LIBSVM\5Z99T\src\ScikitLearnAPI.jl:70
  fit!(::Union{LIBSVM.AbstractSVC, LIBSVM.AbstractSVR}, ::AbstractArray{T,2} where T, !Matched::Array{T,1} where T) at C:\Users\aband\.julia\packages\LIBSVM\5Z99T\src\ScikitLearnAPI.jl:70
  fit!(!Matched::LinearSVC, ::AbstractArray{T,2} where T, !Matched::Array{T,1} where T) at C:\Users\aband\.julia\packages\LIBSVM\5Z99T\src\ScikitLearnAPI.jl:101